<img src="../../images/banners/python-advanced.png" width="600"/>

# <img src="../../images/logos/python.png" width="23"/> An Intro to Threading in Python 


<a class="anchor" id="table_of_contents"></a>
## <img src="../../images/logos/toc.png" width="20"/> Table of Contents  
* [What Is a Thread?](#what_is_a_thread?)
* [Starting a Thread](#starting_a_thread)
    * [Daemon Threads](#daemon_threads)
    * [join() a Thread](#join()_a_thread)
* [Working With Many Threads](#working_with_many_threads)
* [Using a ThreadPoolExecutor](#using_a_threadpoolexecutor)
* [Race Conditions](#race_conditions)
    * [One Thread](#one_thread)
    * [Two Threads](#two_threads)
    * [Why This Isn’t a Silly Example](#why_this_isn’t_a_silly_example)
* [Basic Synchronization Using Lock](#basic_synchronization_using_lock)
* [Deadlock](#deadlock)
* [Producer-Consumer Threading](#producer-consumer_threading)
    * [Producer-Consumer Using Lock](#producer-consumer_using_lock)
    * [Producer-Consumer Using Queue](#producer-consumer_using_queue)
* [Threading Objects](#threading_objects)
    * [Semaphore](#semaphore)
    * [Timer](#timer)
    * [Barrier](#barrier)
* [Conclusion: Threading in Python](#conclusion:_threading_in_python)

---

<p>Python threading allows you to have different parts of your program run concurrently and can simplify your design. If you’ve got some experience in Python and want to speed up your program using threads, then this tutorial is for you!</p>

<p><strong>In this article, you’ll learn:</strong></p>

- What threads are
- How to create threads and wait for them to finish
- How to use a <code>ThreadPoolExecutor</code>
- How to avoid race conditions
- How to use the common tools that Python <code>threading</code> provides


<a class="anchor" id="what_is_a_thread?"></a>
## What Is a Thread? [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>A thread is a separate flow of execution. This means that your program will have two things happening at once. But for most Python 3 implementations the different threads do not actually execute at the same time: they merely appear to.</p>

<p>It’s tempting to think of threading as having two (or more) different processors running on your program, each one doing an independent task at the same time. That’s almost right. The threads may be running on different processors, but they will only be running one at a time. </p>

<p>Getting multiple tasks running simultaneously requires a non-standard implementation of Python, writing some of your code in a different language, or using <code>multiprocessing</code> which comes with some extra overhead.</p>

<p>Because of the way CPython implementation of Python works, threading may not speed up all tasks. This is due to interactions with the <a href="./What Is the Python Global Interpreter Lock (GIL)?.ipynb">GIL</a> that essentially limit one Python thread to run at a time.
The Python Global Interpreter Lock or <a href="https://wiki.python.org/moin/GlobalInterpreterLock">GIL</a>, in simple words, is a mutex (or a lock) that allows only one thread to hold the control of the Python interpreter. You will read more about GIL and how it works in the upcoming sections. <p>This means that only one thread can be in a state of execution at any point in time. The impact of the GIL isn’t visible to developers who execute single-threaded programs, but it can be a performance bottleneck in CPU-bound and multi-threaded code. GIL will be covered in details in the upcoming sections.
</p>

<p>Tasks that spend much of their time waiting for external events are generally good candidates for threading. Problems that require heavy CPU computation and spend little time waiting for external events might not run faster at all. </p>

<p>This is true for code written in Python and running on the standard CPython implementation. If your threads are written in C they have the ability to release the GIL and run concurrently. If you are running on a different Python implementation, check with the documentation too see how it handles threads. </p>

<p>If you are running a standard Python implementation, writing in only Python, and have a CPU-bound problem, you should check out the <code>multiprocessing</code> module instead.</p>

<p>Architecting your program to use threading can also provide gains in design clarity. Most of the examples you’ll learn about in this tutorial are not necessarily going to run faster because they use threads. Using threading in them helps to make the design cleaner and easier to reason about.</p>

<p>So, let’s stop talking about threading and start using it!</p>

<a class="anchor" id="starting_a_thread"></a>
## Starting a Thread [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>Now that you’ve got an idea of what a thread is, let’s learn how to make one. The Python standard library provides <a href="https://docs.python.org/3/library/threading.html"><code>threading</code></a>, which contains most of the primitives you’ll see in this article. <code>Thread</code>, in this module, nicely encapsulates threads, providing a clean interface to work with them.</p>

<p>To start a separate thread, you create a <code>Thread</code> instance and then tell it to <code>.start()</code>:</p>

In [ ]:
import logging
import threading
import time

def thread_function(name):
    logging.info(f"Thread {name}: starting")
    time.sleep(2)
    logging.info(f"Thread {name}: finishing")

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

    logging.info("Main    : before creating thread")
    x = threading.Thread(target=thread_function, args=(1,))
    logging.info("Main    : before running thread")
    x.start()
    logging.info("Main    : wait for the thread to finish")
    # x.join()
    logging.info("Main    : all done")

<p>If you look around the <a href="https://realpython.com/python-logging/">logging</a> statements, you can see that the <code>main</code> section is creating and starting the thread:</p>

In [ ]:
x = threading.Thread(target=thread_function, args=(1,))
x.start()

<p>When you create a <code>Thread</code>, you pass it a function and a list containing the arguments to that function. In this case, you’re telling the <code>Thread</code> to run <code>thread_function()</code> and to pass it <code>1</code> as an argument.</p>

<p>For this article, you’ll use sequential integers as names for your threads. There is <code>threading.get_ident()</code>, which returns a unique name for each thread, but these are usually neither short nor easily readable.</p>

<p><code>thread_function()</code> itself doesn’t do much. It simply logs some messages with a <a href="https://realpython.com/python-sleep/"><code>time.sleep()</code></a> in between them.</p>

<p>When you run this program as it is (with line twenty commented out), the output will look like this:</p>

```sh
$ ./single_thread.py
Main    : before creating thread
Main    : before running thread
Thread 1: starting
Main    : wait for the thread to finish
Main    : all done
Thread 1: finishing
```

<p>You’ll notice that the <code>Thread</code> finished after the <code>Main</code> section of your code did. You’ll come back to why that is and talk about the mysterious line twenty (<code>x.join()</code>) in the next section.</p>

<a class="anchor" id="daemon_threads"></a>
### Daemon Threads [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>In computer science, a <a href="https://en.wikipedia.org/wiki/Daemon_(computing)"><code>daemon</code></a> is a process that runs in the background.</p>

<p>Python <code>threading</code> has a more specific meaning for <code>daemon</code>. A <code>daemon</code> thread will shut down immediately when the program exits. One way to think about these definitions is to consider the <code>daemon</code> thread a thread that runs in the background without worrying about shutting it down.</p>

<p>If a program is running <code>Threads</code> that are not <code>daemons</code>, then the program will wait for those threads to complete before it terminates. <code>Threads</code> that <em>are</em> daemons, however, are just killed wherever they are when the program is exiting.</p>

<p>Let’s look a little more closely at the output of your program above. The last two lines are the interesting bit. When you run the program, you’ll notice that there is a pause (of about 2 seconds) after <a href="https://realpython.com/python-main-function/"><code>__main__</code></a> has printed its <code>all done</code> message and before the thread is finished.</p>

<p>This pause is Python waiting for the non-daemonic thread to complete. When your Python program ends, part of the shutdown process is to clean up the threading routine.</p>

<p>If you look at the <a href="https://github.com/python/cpython/blob/df5cdc11123a35065bbf1636251447d0bfe789a5/Lib/threading.py#L1263">source for Python <code>threading</code></a>, you’ll see that <code>threading._shutdown()</code> walks through all of the running threads and calls <code>.join()</code> on every one that does not have the <code>daemon</code> flag set.</p>

<p>So your program waits to exit because the thread itself is waiting in a sleep. As soon as it has completed and printed the message, <code>.join()</code> will return and the program can exit.</p>

<p>Frequently, this behavior is what you want, but there are other options available to us. Let’s first repeat the program with a <code>daemon</code> thread. You do that by changing how you construct the <code>Thread</code>, adding the <code>daemon=True</code> flag:</p>

In [ ]:
x = threading.Thread(target=thread_function, args=(1,), daemon=True)

<p>When you run the program now, you should see this output:</p>

```sh
$ ./daemon_thread.py
Main    : before creating thread
Main    : before running thread
Thread 1: starting
Main    : wait for the thread to finish
Main    : all done
```

<p>The difference here is that the final line of the output is missing. <code>thread_function()</code> did not get a chance to complete. It was a <code>daemon</code> thread, so when <code>__main__</code> reached the end of its code and the program wanted to finish, the daemon was killed.</p>

<a class="anchor" id="join()_a_thread"></a>
### `join()` a Thread [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>Daemon threads are handy, but what about when you want to wait for a thread to stop? What about when you want to do that and not exit your program? Now let’s go back to your original program and look at that commented out line twenty:</p>

In [ ]:
# x.join()

<p>To tell one thread to wait for another thread to finish, you call <code>.join()</code>. If you uncomment that line, the main thread will pause and wait for the thread <code>x</code> to complete running.</p>

<p>Did you test this on the code with the daemon thread or the regular thread? It turns out that it doesn’t matter. If you <code>.join()</code> a thread, that statement will wait until either kind of thread is finished.</p>

<a class="anchor" id="working_with_many_threads"></a>
## Working With Many Threads [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>The example code so far has only been working with two threads: the main thread and one you started with the <code>threading.Thread</code> object. </p>

<p>Frequently, you’ll want to start a number of threads and have them do interesting work. Let’s start by looking at the harder way of doing that, and then you’ll move on to an easier method.</p>

<p>The harder way of starting multiple threads is the one you already know:</p>

In [ ]:
import logging
import threading
import time

def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(format=format, level=logging.INFO,
                        datefmt="%H:%M:%S")

    threads = list()
    for index in range(3):
        logging.info("Main    : create and start thread %d.", index)
        x = threading.Thread(target=thread_function, args=(index,))
        threads.append(x)
        x.start()

    for index, thread in enumerate(threads):
        logging.info("Main    : before joining thread %d.", index)
        thread.join()
        logging.info("Main    : thread %d done", index)

<p>This code uses the same mechanism you saw above to start a thread, create a <code>Thread</code> object, and then call <code>.start()</code>. The program keeps a list of <code>Thread</code> objects so that it can then wait for them later using <code>.join()</code>.</p>

<p>Running this code multiple times will likely produce some interesting results. Here’s an example output from my machine:</p>

```sh
$ ./multiple_threads.py
Main    : create and start thread 0.
Thread 0: starting
Main    : create and start thread 1.
Thread 1: starting
Main    : create and start thread 2.
Thread 2: starting
Main    : before joining thread 0.
Thread 2: finishing
Thread 1: finishing
Thread 0: finishing
Main    : thread 0 done
Main    : before joining thread 1.
Main    : thread 1 done
Main    : before joining thread 2.
Main    : thread 2 done
```

<p>If you walk through the output carefully, you’ll see all three threads getting started in the order you might expect, but in this case they finish in the opposite order! Multiple runs will produce different orderings. Look for the <code>Thread x: finishing</code> message to tell you when each thread is done.</p>

<p>The order in which threads are run is determined by the operating system and can be quite hard to predict. It may (and likely will) vary from run to run, so you need to be aware of that when you design algorithms that use threading.</p>

<p>Fortunately, Python gives you several primitives that you’ll look at later to help coordinate threads and get them running together. Before that, let’s look at how to make managing a group of threads a bit easier.</p>

<a class="anchor" id="using_a_threadpoolexecutor"></a>
## Using a ThreadPoolExecutor [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>There’s an easier way to start up a group of threads than the one you saw above. It’s called a <code>ThreadPoolExecutor</code>, and it’s part of the standard library in <a href="https://docs.python.org/3/library/concurrent.futures.html"><code>concurrent.futures</code></a> (as of Python 3.2).</p>

<p>The easiest way to create it is as a context manager, using the <a href="https://realpython.com/python-with-statement/"><code>with</code> statement</a> to manage the creation and destruction of the pool.</p>

<p>Here’s the <code>__main__</code> from the last example rewritten to use a <code>ThreadPoolExecutor</code>:</p>

In [ ]:
import concurrent.futures

# [rest of code]

if __name__ == "__main__":
    format = "%(asctime)s: %(message)s"
    logging.basicConfig(
        format=format, level=logging.INFO, datefmt="%H:%M:%S"
    )

    with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
        executor.map(thread_function, range(3))

<p>The code creates a <code>ThreadPoolExecutor</code> as a context manager, telling it how many worker threads it wants in the pool. It then uses <code>.map()</code> to step through an iterable of things, in your case <code>range(3)</code>, passing each one to a thread in the pool. You can think of <code>concurrent.map</code> as just a parallel version of the builtin <code>map</code> function.</p>

<p>The end of the <code>with</code> block causes the <code>ThreadPoolExecutor</code> to do a <code>.join()</code> on each of the threads in the pool. It is <em>strongly</em> recommended that you use <code>ThreadPoolExecutor</code> as a context manager when you can so that you never forget to <code>.join()</code> the threads.</p>

<p>Running your corrected example code will produce output that looks like this:</p>

```sh
$ ./executor.py
Thread 0: starting
Thread 1: starting
Thread 2: starting
Thread 1: finishing
Thread 0: finishing
Thread 2: finishing
```

<p>Again, notice how <code>Thread 1</code> finished before <code>Thread 0</code>. The scheduling of threads is done by the operating system and does not follow a plan that’s easy to figure out.</p>

<a class="anchor" id="race_conditions"></a>
## Race Conditions [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>Before you move on to some of the other features tucked away in Python <code>threading</code>, let’s talk a bit about one of the more difficult issues you’ll run into when writing threaded programs: <a href="https://en.wikipedia.org/wiki/Race_condition">race conditions</a>.</p>

<p>Once you’ve seen what a race condition is and looked at one happening, you’ll move on to some of the primitives provided by the standard library to prevent race conditions from happening.</p>

<p>Race conditions can occur when two or more threads access a shared piece of data or resource. In this example, you’re going to create a large race condition that happens every time, but be aware that most race conditions are not this obvious. Frequently, they only occur rarely, and they can produce confusing results. As you can imagine, <strong>this makes them quite difficult to debug</strong>.</p>

<p>Fortunately, this race condition will happen every time, and you’ll walk through it in detail to explain what is happening.</p>

<p>For this example, you’re going to write a class that updates a database. Okay, you’re not really going to have a database: you’re just going to fake it, because that’s not the point of this article.</p>

<p>Your <code>FakeDatabase</code> will have <code>.__init__()</code> and <code>.update()</code> methods:</p>

In [ ]:
class FakeDatabase:
    def __init__(self):
        self.value = 0

    def update(self, name):
        logging.info(f"Thread {name}: starting update")
        local_copy = self.value
        local_copy += 1
        time.sleep(0.1)
        self.value = local_copy
        logging.info(f"Thread {name}: finishing update")

<p><code>FakeDatabase</code> is keeping track of a single number: <code>.value</code>. This is going to be the shared data on which you’ll see the race condition.</p>

<p><code>.__init__()</code> simply initializes <code>.value</code> to zero. So far, so good.</p>

<p><code>.update()</code> looks a little strange. It’s simulating reading a value from a database, doing some computation on it, and then writing a new value back to the database.</p>

<p>In this case, reading from the database just means copying <code>.value</code> to a local variable. The computation is just to add one to the value and then <code>.sleep()</code> for a little bit. Finally, it writes the value back by copying the local value back to <code>.value</code>.</p>

<p>Here’s how you’ll use this <code>FakeDatabase</code>:</p>

In [ ]:
format = "%(asctime)s: %(message)s"
logging.basicConfig(format=format, level=logging.INFO, datefmt="%H:%M:%S")

database = FakeDatabase()
logging.info(f"Testing update. Starting value is {database.value}.")
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for ® in range(2):
        executor.submit(database.update, index)
logging.info(f"Testing update. Ending value is {database.value}.")

<p>The program creates a <code>ThreadPoolExecutor</code> with two threads and then calls <code>.submit()</code> on each of them, telling them to run <code>database.update()</code>.</p>

<p><code>.submit()</code> has a signature that allows both positional and named arguments to be passed to the function running in the thread:</p>

In [ ]:
.submit(function, *args, **kwargs)

<p>In the usage above, <code>index</code> is passed as the first and only positional argument to <code>database.update()</code>. You’ll see later in this article where you can pass multiple arguments in a similar manner.</p>

<p>Since each thread runs <code>.update()</code>, and <code>.update()</code> adds one to <code>.value</code>, you might expect <code>database.value</code> to be <code>2</code> when it’s printed out at the end. But you wouldn’t be looking at this example if that was the case. If you run the above code, the output looks like this:</p>

```sh
$ ./racecond.py
Testing unlocked update. Starting value is 0.
Thread 0: starting update
Thread 1: starting update
Thread 0: finishing update
Thread 1: finishing update
Testing unlocked update. Ending value is 1.
```

<p>You might have expected that to happen, but let’s look at the details of what’s really going on here, as that will make the solution to this problem easier to understand.</p>

<a class="anchor" id="one_thread"></a>
### One Thread [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>Before you dive into this issue with two threads, let’s step back and talk a bit about some details of how threads work.</p>

<p>You won’t be diving into all of the details here, as that’s not important at this level. We’ll also be simplifying a few things in a way that won’t be technically accurate but will give you the right idea of what is happening.</p>

<p>When you tell your <code>ThreadPoolExecutor</code> to run each thread, you tell it which function to run and what parameters to pass to it: <code>executor.submit(database.update, index)</code>. </p>

<p>The result of this is that each of the threads in the pool will call <code>database.update(index)</code>. Note that <code>database</code> is a reference to the one <code>FakeDatabase</code> object created in <code>__main__</code>. Calling <code>.update()</code> on that object calls an <a href="https://realpython.com/instance-class-and-static-methods-demystified/">instance method</a> on that object.</p>

<p>Each thread is going to have a reference to the same <code>FakeDatabase</code> object, <code>database</code>. Each thread will also have a unique value, <code>index</code>, to make the logging statements a bit easier to read:</p>

<img src="images/an-intro-to-threading-in-python/intro-threading-shared-database.267a5d8c6aa1.png" width="600px">

<p>When the thread starts running <code>.update()</code>, it has its own version of all of the data <strong>local</strong> to the function. In the case of <code>.update()</code>, this is <code>local_copy</code>. This is definitely a good thing. Otherwise, two threads running the same function would always confuse each other. It means that all variables that are scoped (or local) to a function are <strong>thread-safe</strong>.</p>

<p>Now you can start walking through what happens if you run the program above with a single thread and a single call to <code>.update()</code>.</p>

<p>The image below steps through the execution of <code>.update()</code> if only a single thread is run. The statement is shown on the left followed by a diagram showing the values in the thread’s <code>local_copy</code> and the shared <code>database.value</code>:</p>

<img src="images/an-intro-to-threading-in-python/intro-threading-single-thread.6a11288bc199.png" width="600px">

<p>The diagram is laid out so that time increases as you move from top to bottom. It begins when <code>Thread 1</code> is created and ends when it is terminated.</p>

<p>When <code>Thread 1</code> starts, <code>FakeDatabase.value</code> is zero. The first line of code in the method, <code>local_copy = self.value</code>, copies the value zero to the local variable. Next it increments the value of <code>local_copy</code> with the <code>local_copy += 1</code> statement. You can see <code>.value</code> in <code>Thread 1</code> getting set to one.</p>

<p>Next <code>time.sleep()</code> is called, which makes the current thread pause and allows other threads to run. Since there is only one thread in this example, this has no effect.</p>

<p>When <code>Thread 1</code> wakes up and continues, it copies the new value from <code>local_copy</code> to <code>FakeDatabase.value</code>, and then the thread is complete. You can see that <code>database.value</code> is set to one.</p>

<p>So far, so good. You ran <code>.update()</code> once and <code>FakeDatabase.value</code> was incremented to one.</p>

<a class="anchor" id="two_threads"></a>
### Two Threads [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>Getting back to the race condition, the two threads will be running concurrently but not at the same time. They will each have their own version of <code>local_copy</code> and will each point to the same <code>database</code>. It is this shared <code>database</code> object that is going to cause the problems.</p>

<p>The program starts with <code>Thread 1</code> running <code>.update()</code>:</p>

<img src="images/an-intro-to-threading-in-python/intro-threading-two-threads-part1.c1c0e65a8481.png" width="600px">

<p>When <code>Thread 1</code> calls <code>time.sleep()</code>, it allows the other thread to start running. This is where things get interesting.</p>

<p><code>Thread 2</code> starts up and does the same operations. It’s also copying <code>database.value</code> into its private <code>local_copy</code>, and this shared <code>database.value</code> has not yet been updated:</p>

<img src="images/an-intro-to-threading-in-python/intro-threading-two-threads-part2.df42d4fbfe21.png" width="600px">

<p>When <code>Thread 2</code> finally goes to sleep, the shared <code>database.value</code> is still unmodified at zero, and both private versions of <code>local_copy</code> have the value one.</p>

<p><code>Thread 1</code> now wakes up and saves its version of <code>local_copy</code> and then terminates, giving <code>Thread 2</code> a final chance to run. <code>Thread 2</code> has no idea that <code>Thread 1</code> ran and updated <code>database.value</code> while it was sleeping. It stores <em>its</em> version of <code>local_copy</code> into <code>database.value</code>, also setting it to one:</p>

<img src="images/an-intro-to-threading-in-python/intro-threading-two-threads-part3.18576920f88f.png" width="600px">

<p>The two threads have interleaving access to a single shared object, overwriting each other’s results. Similar race conditions can arise when one thread frees memory or closes a file handle before the other thread is finished accessing it.</p>

<a class="anchor" id="why_this_isn’t_a_silly_example"></a>
### Why This Isn’t a Silly Example [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>The example above is contrived to make sure that the race condition happens every time you run your program. Because the operating system can swap out a thread at any time, it is possible to interrupt a statement like <code>x = x + 1</code> after it has read the value of <code>x</code> but before it has written back the incremented value.</p>

<p>The details of how this happens are quite interesting, but not needed for the rest of this article, so feel free to skip over the section below.</p>

#### How Does This Really Work [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

The code above isn’t quite as out there as you might originally have thought. It was designed to force a race condition every time you run it, but that makes it much easier to solve than most race conditions.

There are two things to keep in mind when thinking about race conditions:

1. Even an operation like x += 1 takes the processor many steps. Each of these steps is a separate instruction to the processor.
2. The operating system can swap which thread is running at any time. A thread can be swapped out after any of these small instructions. This means that a thread can be put to sleep to let another thread run in the middle of a Python statement.

Let’s look at this in detail. The REPL below shows a function that takes a parameter and increments it:

In [2]:
def inc(x):
    x += 1

In [3]:
import dis

In [4]:
dis.dis(inc)

  2           0 LOAD_FAST                0 (x)
              2 LOAD_CONST               1 (1)
              4 INPLACE_ADD
              6 STORE_FAST               0 (x)
              8 LOAD_CONST               0 (None)
             10 RETURN_VALUE


The REPL example uses [dis](https://docs.python.org/3/library/dis.html) from the Python standard library to show the smaller steps that the processor does to implement your function. It does a `LOAD_FAST` of the data value `x`, it does a `LOAD_CONST 1`, and then it uses the `INPLACE_ADD` to add those values together.

We’re stopping here for a specific reason. This is the point in `.update()` above where `time.sleep()` forced the threads to switch. It is entirely possible that, every once in while, the operating system would switch threads at that exact point even without `sleep()`, but the call to `sleep()` makes it happen every time.

As you learned above, the operating system can swap threads at any time. You’ve walked down this listing to the statement marked 4. If the operating system swaps out this thread and runs a different thread that also modifies `x`, then when this thread resumes, it will overwrite `x` with an incorrect value.

Technically, this example won’t have a race condition because `x` is local to `inc()`. It does illustrate how a thread can be interrupted during a single Python operation, however. The same `LOAD`, `MODIFY`, `STORE` set of operations also happens on global and shared values. You can explore with the `dis` module and prove that yourself.

It’s rare to get a race condition like this to occur, but remember that an infrequent event taken over millions of iterations becomes likely to happen. **The rarity of these race conditions makes them much, much harder to debug than regular bugs.**

Now back to your regularly scheduled tutorial!

<p>Now that you’ve seen a race condition in action, let’s find out how to solve them!</p>

<a class="anchor" id="basic_synchronization_using_lock"></a>
## Basic Synchronization Using Lock [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>There are a number of ways to avoid or solve race conditions. You won’t look at all of them here, but there are a couple that are used frequently. Let’s start with <code>Lock</code>.</p>

<p>To solve your race condition above, you need to find a way to allow only one thread at a time into the read-modify-write section of your code. The most common way to do this is called <code>Lock</code> in Python. In some other languages this same idea is called a <code>mutex</code>. Mutex comes from MUTual EXclusion, which is exactly what a <code>Lock</code> does.</p>

<p>A <code>Lock</code> is an object that acts like a hall pass. Only one thread at a time can have the <code>Lock</code>. Any other thread that wants the <code>Lock</code> must wait until the owner of the <code>Lock</code> gives it up.</p>

<p>The basic functions to do this are <code>.acquire()</code> and <code>.release()</code>. A thread will call <code>my_lock.acquire()</code> to get the lock. If the lock is already held, the calling thread will wait until it is released. There’s an important point here. If one thread gets the lock but never gives it back, your program will be stuck. You’ll read more about this later.</p>

<p>Fortunately, Python’s <code>Lock</code> will also operate as a context manager, so you can use it in a <code>with</code> statement, and it gets released automatically when the <code>with</code> block exits for any reason.</p>

<p>Let’s look at the <code>FakeDatabase</code> with a <code>Lock</code> added to it. The calling function stays the same:</p>

In [ ]:
class FakeDatabase:
    def __init__(self):
        self.value = 0
        self._lock = threading.Lock()

    def locked_update(self, name):
        logging.info("Thread %s: starting update", name)
        logging.debug("Thread %s about to lock", name)
        with self._lock:
            logging.debug("Thread %s has lock", name)
            local_copy = self.value
            local_copy += 1
            time.sleep(0.1)
            self.value = local_copy
            logging.debug("Thread %s about to release lock", name)
        logging.debug("Thread %s after release", name)
        logging.info("Thread %s: finishing update", name)

<p>Other than adding a bunch of debug logging so you can see the locking more clearly, the big change here is to add a member called <code>._lock</code>, which is a <code>threading.Lock()</code> object. This <code>._lock</code> is initialized in the unlocked state and locked and released by the <code>with</code> statement.</p>

<p>It’s worth noting here that the thread running this function will hold on to that <code>Lock</code> until it is completely finished updating the database. In this case, that means it will hold the <code>Lock</code> while it copies, updates, sleeps, and then writes the value back to the database.</p>

<p>If you run this version with logging set to warning level, you’ll see this:</p>

```sh
$ ./fixrace.py
Testing locked update. Starting value is 0.
Thread 0: starting update
Thread 1: starting update
Thread 0: finishing update
Thread 1: finishing update
Testing locked update. Ending value is 2.
```

<p>Look at that. Your program finally works!</p>

<p>You can turn on full logging by setting the level to <code>DEBUG</code> by adding this statement after you configure the logging output in <code>__main__</code>:</p>

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)

<p>Running this program with <code>DEBUG</code> logging turned on looks like this:</p>

```sh
$ ./fixrace.py
Testing locked update. Starting value is 0.
Thread 0: starting update
Thread 0 about to lock
Thread 0 has lock
Thread 1: starting update
Thread 1 about to lock
Thread 0 about to release lock
Thread 0 after release
Thread 0: finishing update
Thread 1 has lock
Thread 1 about to release lock
Thread 1 after release
Thread 1: finishing update
Testing locked update. Ending value is 2.
```

<p>In this output you can see <code>Thread 0</code> acquires the lock and is still holding it when it goes to sleep. <code>Thread 1</code> then starts and attempts to acquire the same lock. Because <code>Thread 0</code> is still holding it, <code>Thread 1</code> has to wait. This is the mutual exclusion that a <code>Lock</code> provides.</p>

<p>Many of the examples in the rest of this article will have <code>WARNING</code> and <code>DEBUG</code> level logging. We’ll generally only show the <code>WARNING</code> level output, as the <code>DEBUG</code> logs can be quite lengthy. Try out the programs with the logging turned up and see what they do.</p>

<a class="anchor" id="deadlock"></a>
## Deadlock [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>Before you move on, you should look at a common problem when using <code>Locks</code>. As you saw, if the <code>Lock</code> has already been acquired, a second call to <code>.acquire()</code> will wait until the thread that is holding the <code>Lock</code> calls <code>.release()</code>. What do you think happens when you run this code:</p>

In [ ]:
import threading

l = threading.Lock()
print("before first acquire")
l.acquire()
print("before second acquire")
l.acquire()
print("acquired lock twice")

<p>When the program calls <code>l.acquire()</code> the second time, it hangs waiting for the <code>Lock</code> to be released. In this example, you can fix the deadlock by removing the second call, but deadlocks usually happen from one of two subtle things:</p>

1. An implementation bug where a <code>Lock</code> is not released properly
2. A design issue where a utility function needs to be called by functions that might or might not already have the <code>Lock</code>


<p>The first situation happens sometimes, but using a <code>Lock</code> as a context manager greatly reduces how often. It is recommended to write code whenever possible to make use of context managers, as they help to avoid situations where an exception skips you over the <code>.release()</code> call.</p>

<p>The design issue can be a bit trickier in some languages. Thankfully, Python threading has a second object, called <code>RLock</code>, that is designed for just this situation. It allows a thread to <code>.acquire()</code> an <code>RLock</code> multiple times before it calls <code>.release()</code>. That thread is still required to call <code>.release()</code> the same number of times it called <code>.acquire()</code>, but it should be doing that anyway.</p>

<p><code>Lock</code> and <code>RLock</code> are two of the basic tools used in threaded programming to prevent race conditions. There are a few other that work in different ways. Before you look at them, let’s shift to a slightly different problem domain.</p>

<a class="anchor" id="threading_objects"></a>
## Threading Objects [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>There are a few more primitives offered by the Python <code>threading</code> module. While you didn’t need these for the examples above, they can come in handy in different use cases, so it’s good to be familiar with them.</p>

<a class="anchor" id="semaphore"></a>
### Semaphore [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>The first Python <code>threading</code> object to look at is <code>threading.Semaphore</code>. A <code>Semaphore</code> is a counter with a few special properties. The first one is that the counting is atomic. This means that there is a guarantee that the operating system will not swap out the thread in the middle of incrementing or decrementing the counter.</p>

<p>The internal counter is incremented when you call <code>.release()</code> and decremented when you call <code>.acquire()</code>.</p>

<p>The next special property is that if a thread calls <code>.acquire()</code> when the counter is zero, that thread will block until a different thread calls <code>.release()</code> and increments the counter to one.</p>

<p>Semaphores are frequently used to protect a resource that has a limited capacity. An example would be if you have a pool of connections and want to limit the size of that pool to a specific number.</p>

<a class="anchor" id="timer"></a>
### Timer [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>A <code>threading.Timer</code> is a way to schedule a function to be called after a certain amount of time has passed. You create a <code>Timer</code> by passing in a number of seconds to wait and a function to call:</p>

In [ ]:
t = threading.Timer(30.0, my_function)

<p>You start the <code>Timer</code> by calling <code>.start()</code>. The function will be called on a new thread at some point after the specified time, but be aware that there is no promise that it will be called exactly at the time you want.</p>

<p>If you want to stop a <code>Timer</code> that you’ve already started, you can cancel it by calling <code>.cancel()</code>. Calling <code>.cancel()</code> after the <code>Timer</code> has triggered does nothing and does not produce an exception.</p>

<p>A <code>Timer</code> can be used to prompt a user for action after a specific amount of time. If the user does the action before the <code>Timer</code> expires, <code>.cancel()</code> can be called.</p>

<a class="anchor" id="barrier"></a>
### Barrier [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>A <code>threading.Barrier</code> can be used to keep a fixed number of threads in sync. When creating a <code>Barrier</code>, the caller must specify how many threads will be synchronizing on it. Each thread calls <code>.wait()</code> on the <code>Barrier</code>. They all will remain blocked until the specified number of threads are waiting, and then the are all released at the same time.</p>

<p>Remember that threads are scheduled by the operating system so, even though all of the threads are released simultaneously, they will be scheduled to run one at a time.</p>

<p>One use for a <code>Barrier</code> is to allow a pool of threads to initialize themselves. Having the threads wait on a <code>Barrier</code> after they are initialized will ensure that none of the threads start running before all of the threads are finished with their initialization.</p>

<a class="anchor" id="conclusion:_threading_in_python"></a>
## Conclusion: Threading in Python [<img src="../../images/logos/back_to_top.png" width="22" align= "center"/>](#table_of_contents)

<p>You’ve now seen much of what Python <code>threading</code> has to offer and some examples of how to build threaded programs and the problems they solve. You’ve also seen a few instances of the problems that arise when writing and debugging threaded programs.</p>

<p>If you’d like to explore other options for concurrency in Python, check out <a href="https://realpython.com/python-concurrency/">Speed Up Your Python Program With Concurrency</a>.</p>

<p>If you’re interested in doing a deep dive on the <code>asyncio</code> module, go read <a href="https://realpython.com/async-io-python/">Async IO in Python: A Complete Walkthrough</a>.</p>

<p>Whatever you do, you now have the information and confidence you need to write programs using Python threading!</p>

<p><em>Special thanks to reader JL Diaz for helping to clean up the introduction.</em></p>